In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Start Scraping

In [3]:
mainweb = 'https://what-if.xkcd.com'

In [4]:
r = requests.get(mainweb, auth=('user', 'pass'))
soup = BeautifulSoup(r.content, 'html.parser')

In [5]:
lastpage = soup.find_all('a', href=True)
lastpage = lastpage[1].get('href')
lastpage = re.sub(r'/',r"",lastpage)
lastpage = int(lastpage) + 1
lastpage

157

In [6]:
idquestion = []
judul = []
pertanyaan = []
penanya = []
jawaban = []
idq1 = []
titleill = []
srcill = []
idq2 = []
textreference = []
for i in tqdm(range(1,lastpage+1,1)):
    r = requests.get(mainweb +'/'+ str(i), auth=('user', 'pass'))
    soup = BeautifulSoup(r.content, 'html.parser')

    idquestion.append(i)
    
    #Get Title
    try:
        judul.append(soup.find('h1').get_text())
    except:
        judul.append('')
    
    #Get Query
    try :
        currency = soup.find_all('p', id='question')
        per = ''
        for elm in currency:
            per = per +' ; '+ elm.get_text()
        per = re.sub(r'^\s*\;\s*',r'',per) #Delete simbol ; yang ada di depan string
        per = re.sub(r'\—',r'',per) #Delete —
        pertanyaan.append(per)
    except :
        pertanyaan.append('')
    
    #Get Inquirer
    try :
        currency = soup.find_all('p', id='attribute')
        pen = ''
        for elm in currency:
            pen = pen +' ; '+ elm.get_text()
        pen = re.sub(r'^\s*\;\s*',r'',pen) #Delete simbol ; yang ada di depan string
        pen = re.sub(r'\—\s*',r'',pen) #Delete —
        pen = re.sub(r'^\–\s*',r'',pen) #Delete -
        penanya.append(pen)
    except :
        penanya.append('')
    
    #Get Text
    try :
        currency = soup.find_all('p')
        text = ''
        for elm in currency[1].find_next_siblings('p'):
            text = text + elm.get_text()
        text = re.sub(r'\[\d+\]',r" ",text) #Delete angka referensi
        text = re.sub(r'(\\\[.{1,100}\\\])',r"",text) #Delete Rumus (hanya sebagian)
        text = re.sub(r'(\\.*\n*.*\n*.*\\)',r"",text) #Delete Rumus (hanya sebagian)
        jawaban.append(text)
    except:
        jawaban.append('')
    
    #Get Illustration
    ill = soup.find_all('img', class_='illustration')
    for j in ill :
        idq1.append(i)
        title = str(j.get('title'))
        src = str(j.get('src'))
        titleill.append(title)
        srcill.append(mainweb + src)
    
    #Get Reference
    ref = soup.find_all('span', class_='refbody')
    for k in ref:
        idq2.append(i)
        textref = str(k.get_text())
        textreference.append(textref)

100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [03:32<00:00,  1.50it/s]


In [7]:
question = pd.DataFrame()
illustration = pd.DataFrame()
reference = pd.DataFrame()

In [8]:
for i in range(len(judul)):
    question.loc[i,'idquestion'] = idquestion[i]
    question.loc[i,'titlequestion'] = judul[i]
    question.loc[i,'query'] = pertanyaan[i]
    question.loc[i,'inquirer'] = penanya[i]
    question.loc[i,'answer'] = jawaban[i]
    question.loc[i,'length_word'] = len(question.loc[i,'answer'].split())
    question.loc[i,'length_char'] = len(question.loc[i,'answer'])

In [9]:
question

,idquestion,titlequestion,query,inquirer,answer,length_word,length_char
0,1.0,Relativistic Baseball,What would happen if you tried to hit a baseba...,- Ellen McManis,Let’s set aside the question of how we got the...,677.0,3962.0
1,2.0,SAT Guessing,What if everyone who took the SAT guessed on e...,Rob Balder,None.The SAT is a standardized test given to h...,343.0,2084.0
2,3.0,Yoda,How much Force power can Yoda output?,Ryan Finnie,I’m going to—of course—ignore the prequels.The...,565.0,3224.0
3,4.0,A Mole of Moles,What would happen if you were to gather a mole...,Sean Rice,"Things get a bit gruesome.First, some definiti...",994.0,5884.0
4,5.0,Robot Apocalypse,What if there was a robot apocalypse? How long...,Rob Lombino,"Before I answer this question, let me give you...",1167.0,7044.0
5,6.0,Glass Half Empty,"What if a glass of water was, all of a sudden,...",Vittorio Iacovella,The pessimist is probably more right about how...,898.0,5241.0
6,7.0,Everybody Out,Is there enough energy to move the entire curr...,Adam,There are a bunch of science fiction movies wh...,1039.0,6055.0
7,8.0,Everybody Jump,What would happen if everyone on earth stood a...,Thomas Bennett (and many others),This is one of the most popular questions subm...,616.0,3714.0
8,9.0,Soul Mates,What if everyone actually had only one soul ma...,Benjamin Staffin,What a nightmare that would be.There are a lot...,890.0,4982.0
9,10.0,Cassini,What would the world be like if the land masse...,Socke,It would profoundly alter our biosphere in gen...,1742.0,10574.0


In [10]:
for i in range(len(titleill)):
    illustration.loc[i,'idillustration'] = i
    illustration.loc[i,'idquestion'] = idq1[i]
    illustration.loc[i,'titleillustration'] = titleill[i]
    illustration.loc[i,'src'] = srcill[i]

In [11]:
illustration

,idillustration,idquestion,titleillustration,src
0,0.0,1.0,pitcher throwing ball,https://what-if.xkcd.com/imgs/a/1/01.png
1,1.0,1.0,fusion illustration,https://what-if.xkcd.com/imgs/a/1/02.png
2,2.0,1.0,fusion zone of baseball,https://what-if.xkcd.com/imgs/a/1/03.png
3,3.0,1.0,t=30 nanoseconds,https://what-if.xkcd.com/imgs/a/1/04.png
4,4.0,1.0,mushroom cloud,https://what-if.xkcd.com/imgs/a/1/05.png
5,5.0,2.0,multiple choice question bubbles,https://what-if.xkcd.com/imgs/a/2/01.png
6,6.0,2.0,lightning strikes,https://what-if.xkcd.com/imgs/a/2/03.png
7,7.0,3.0,Yoda controlling four wind turbines,https://what-if.xkcd.com/imgs/a/3/01.png
8,8.0,3.0,illustration of an X-Wing and F-22,https://what-if.xkcd.com/imgs/a/3/02.png
9,9.0,3.0,character watching star wars for scientific re...,https://what-if.xkcd.com/imgs/a/3/04.png


In [12]:
for i in range(len(textreference)):
    reference.loc[i,'idreference'] = i
    reference.loc[i,'idquestion'] = idq2[i]
    reference.loc[i,'text'] = textreference[i]

In [13]:
reference

,idreference,idquestion,text
0,0.0,2.0,"Cooper, Mary Ann, MD. ""Disability, Not Death i..."
1,1.0,2.0,http://www.nws.noaa.gov/om/hazstats/light08.pdf
2,2.0,51.0,Super Mario 1-1 speed run
3,3.0,51.0,Sprint ring cycle
4,4.0,51.0,Glide data
5,5.0,51.0,"Jump. Fly. Land., Air & Space"
6,6.0,51.0,"Prof. Dr. Herrligkoffer, The East Pillar of Na..."
7,7.0,51.0,The Guestroom - Dr Glenn Singleman and Heather...
8,8.0,51.0,Highest BASE jump: Valery Rozov breaks Guinnes...
9,9.0,51.0,"Dean Potter, Above It All"


In [14]:
alltable = question.merge(illustration, on='idquestion', how='inner')
alltable = alltable.merge(reference, on='idquestion', how='inner')

In [15]:
# Create a Pandas Excel
writer = pd.ExcelWriter('tugas1_0102516002.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
question.to_excel(writer, sheet_name='Question',index=False)
illustration.to_excel(writer, sheet_name='Illustration',index=False)
reference.to_excel(writer, sheet_name='Reference',index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Top 5 Jawaban Terpanjang

#### By word

In [16]:
question.sort_values(by='length_word', ascending=False).head()

,idquestion,titlequestion,query,inquirer,answer,length_word,length_char
156,157.0,Earth-Moon Fire Pole,My son (5y) asked me today: If there were a ki...,"Ramon Schönborn, Germany","First, let's get a few things out of the way:I...",2209.0,12416.0
66,67.0,Expanding Earth,How long would it take for people to notice th...,Dennis O'Donnell,"The Earth is not, currently, expanding. Yes, I...",1828.0,11155.0
9,10.0,Cassini,What would the world be like if the land masse...,Socke,It would profoundly alter our biosphere in gen...,1742.0,10574.0
12,13.0,Laser Pointer,If every person on Earth aimed a laser pointer...,Peter Lipowicz,Not if we use regular laser pointers.The first...,1581.0,9258.0
27,28.0,Steak Drop,From what height would you need to drop a stea...,Alex Lahey,I hope you like your steaks Pittsburgh Rare. A...,1448.0,8465.0


#### By char

In [17]:
question.sort_values(by='length_char', ascending=False).head()

,idquestion,titlequestion,query,inquirer,answer,length_word,length_char
156,157.0,Earth-Moon Fire Pole,My son (5y) asked me today: If there were a ki...,"Ramon Schönborn, Germany","First, let's get a few things out of the way:I...",2209.0,12416.0
66,67.0,Expanding Earth,How long would it take for people to notice th...,Dennis O'Donnell,"The Earth is not, currently, expanding. Yes, I...",1828.0,11155.0
9,10.0,Cassini,What would the world be like if the land masse...,Socke,It would profoundly alter our biosphere in gen...,1742.0,10574.0
12,13.0,Laser Pointer,If every person on Earth aimed a laser pointer...,Peter Lipowicz,Not if we use regular laser pointers.The first...,1581.0,9258.0
27,28.0,Steak Drop,From what height would you need to drop a stea...,Alex Lahey,I hope you like your steaks Pittsburgh Rare. A...,1448.0,8465.0


### Rata-rata dan Simpangan Baku dari Panjang Jawaban

#### By word

In [18]:
print('Rata-rata dari panjang jawaban = {mean:.2f}'.format(mean = question['length_word'].mean(axis = 0, skipna = False)))

Rata-rata dari panjang jawaban = 770.74


In [19]:
print('Simpangan Baku dari panjang jawaban = {std:.2f}'.format(std = question['length_word'].std(axis = 0, skipna = False)))

Simpangan Baku dari panjang jawaban = 308.49


#### By char

In [20]:
print('Rata-rata dari panjang jawaban = {mean:.2f}'.format(mean = question['length_char'].mean(axis = 0, skipna = False)))

Rata-rata dari panjang jawaban = 4497.43


In [21]:
print('Simpangan Baku dari panjang jawaban = {std:.2f}'.format(std = question['length_char'].std(axis = 0, skipna = False)))

Simpangan Baku dari panjang jawaban = 1815.63


### Rata-rata dan Simpangan Baku dari jumlah referensi per pertanyaan

In [22]:
dummy = pd.DataFrame()
for i in range(1,157+1,1):
    dummy.loc[i,'idquestion'] = i

In [23]:
refgroup = reference.groupby(['idquestion']).count().reset_index()
refgroup = refgroup.merge(dummy, on='idquestion', how='right')
refgroup = refgroup.fillna(0)
refgroup = refgroup.sort_values(by='idquestion', ascending=True).reset_index(drop=True)

In [24]:
print('Rata-rata dari jumlah referensi per pertanyaan = {mean:.2f}'.format(mean = refgroup['idreference'].mean(axis = 0, skipna = False)))

Rata-rata dari jumlah referensi per pertanyaan = 4.03


In [25]:
print('Simpangan Baku dari jumlah referensi per pertanyaan = {std:.2f}'.format(std = refgroup['idreference'].std(axis = 0, skipna = False)))

Simpangan Baku dari jumlah referensi per pertanyaan = 3.99


### Korelasi antara panjang pertanyaan dan panjang jawaban

In [26]:
corr = pd.DataFrame()

#### By word

In [27]:
for i in range(len(question)):
    corr.loc[i,'length_query'] = len(question.loc[i,'query'].split())
    corr.loc[i,'length_answer'] = len(question.loc[i,'answer'].split())

In [28]:
corr.corr()

,length_query,length_answer
length_query,1.000000,0.077928
length_answer,0.077928,1.000000


#### By char

In [29]:
for i in range(len(question)):
    corr.loc[i,'length_query'] = len(question.loc[i,'query'])
    corr.loc[i,'length_answer'] = len(question.loc[i,'answer'])

In [30]:
corr.corr()

,length_query,length_answer
length_query,1.000000,0.068759
length_answer,0.068759,1.000000


### Korelasi antara panjang jawaban dan jumlah ilustrasi

In [31]:
corr2 = pd.DataFrame()

In [32]:
illgroup = illustration.groupby(['idquestion']).count().reset_index()

#### By word

In [33]:
for i in range(len(question)):
    corr2.loc[i,'length_answer'] = question.loc[i,'length_word']
    corr2.loc[i,'total_illustration'] = illgroup.loc[i,'idillustration']

In [34]:
corr2.corr()

,length_answer,total_illustration
length_answer,1.000000,0.424878
total_illustration,0.424878,1.000000


#### By char

In [35]:
for i in range(len(question)):
    corr2.loc[i,'length_answer'] = question.loc[i,'length_char']
    corr2.loc[i,'total_illustration'] = illgroup.loc[i,'idillustration']

In [36]:
corr2.corr()

,length_answer,total_illustration
length_answer,1.000000,0.410577
total_illustration,0.410577,1.000000


### 10 Kata yang sering muncul dari jawaban

In [37]:
X = question['answer']
vec = CountVectorizer(stop_words='english').fit(X)
bag_of_words = vec.transform(X)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
df = pd.DataFrame(words_freq, columns=['Word', 'Frequency'])
df.head(10)

,Word,Frequency
0,water,410
1,earth,396
2,like,320
3,just,278
4,probably,267
5,air,252
6,speed,237
7,time,221
8,surface,211
9,energy,203
